In [1]:
import csv  
import random
import datetime
import pandas as pd

In [2]:
services_header = ['row_id', 'subject_id', 'hadm_id', 'transfertime', 'prev_serv', 'curr_serv']

In [3]:
patients_df = pd.read_csv('patients_data.csv')
patients_df.head(5)

,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,1,F,11-29-1969,None,None,None,0
1,2,M,11-22-1987,None,None,None,0
2,3,F,04-14-1965,None,None,None,0
3,4,M,09-14-1972,06-03-2015,06-03-2015,None,1
4,5,M,08-13-1977,07-01-2020,07-01-2020,07-01-2020,1


In [4]:
admissions_df = pd.read_csv('admissions_data.csv')
admissions_df.head(5)

,Unnamed: 0,subject_id,hadm_id,admit_date,discharge_date,admission_type,admission_location,discharge_location,insurance,religion,marital_status,ethnicity,diagnosis,language
0,0,1,5815,12-06-2017,07-07-2020,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,CATHOLIC,SINGLE,UNABLE TO OBTAIN,SEIZURE,ENGL
1,1,2,5816,10-14-2035,01-19-2036,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,NOT SPECIFIED,WIDOWED,WHITE,HYPONATREMIA;URINARY TRACT INFECTION,RUSS
2,2,3,5817,07-14-1974,02-29-1976,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,UNOBTAINABLE,WIDOWED,WHITE,PNEUMONIA/HYPOGLCEMIA/SYNCOPE,ENGL
3,3,4,5818,04-26-1989,08-24-1992,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,CATHOLIC,MARRIED,WHITE,HUMERAL FRACTURE,ENGL
4,4,5,5819,08-03-1983,07-16-1986,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NOT SPECIFIED,MARRIED,WHITE,FEVER,ENGL


In [5]:
services = ['CMED', 'CSURG', 'DENT', 'ENT', 'GU', 'GYN', 'MED', 'NB', 'NBB', 'NMED', 'NSURG', 'OBS', 'ORTHO', 'OMED', 'PSURG', 'PSYCH', 'SURG', 'TRAUM', 'TSURG', 'VSURG']

In [6]:
def random_date(start, end):
  """
  This function will return a random datetime between two datetime 
  objects.
  """
  # print(start, end)
  delta = end - start
  int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
  if int_delta<=0:
    return ''
  # print(int_delta)
  random_second = random.randrange(int_delta)
  randDate = start + datetime.timedelta(seconds=random_second)
  return randDate.strftime("%m-%d-%Y %H:%M:%S")

def calcDod(dob):
  dod = datetime.datetime.strptime(dob, "%m-%d-%Y")+datetime.timedelta(days = 365*60)
  return dod.strftime("%m-%d-%Y")

In [7]:
patients_df['dod'] = patients_df.dod.astype(str)
patients_df.head()

,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,1,F,11-29-1969,None,None,None,0
1,2,M,11-22-1987,None,None,None,0
2,3,F,04-14-1965,None,None,None,0
3,4,M,09-14-1972,06-03-2015,06-03-2015,None,1
4,5,M,08-13-1977,07-01-2020,07-01-2020,07-01-2020,1


In [8]:
services_df = pd.DataFrame(columns=services_header)

In [9]:
import math

rowId = 1
for i,r in admissions_df.iterrows():
  prevServ = 'NONE'
  currServ = ''
  servicesCount = random.randint(1,3)
  subject_id = int(r['subject_id'])
  dob = patients_df.iloc[subject_id-1]['dob']
  admitTime = r['admit_date'] + ' 00:00:00'
  dischargeTime = r['discharge_date'] + ' 00:00:00'
  prevTransferTime = dob + ' 00:00:00'
  transferTime = admitTime

  # dod = patients_df.iloc[subject_id-1]['dod']
  # # print(i, dod, type(dod))
  # if(dod == 'nan'):
  #   dod = calcDod(dob)
  # # print(i, dod, type(dod))
  # finalDod = dod + ' 00:00:00'

  for _ in range(servicesCount):
    lst = []
    currServ = random.choice(services)
    lst.append(rowId)
    lst.append(subject_id)
    lst.append(r['hadm_id'])
    lst.append(transferTime)
    lst.append(prevServ)
    lst.append(currServ)
    prevTransferTime = transferTime
    d1 = datetime.datetime.strptime(prevTransferTime, '%m-%d-%Y %H:%M:%S')
    d2 = datetime.datetime.strptime(dischargeTime, '%m-%d-%Y %H:%M:%S')
    transferTime = random_date(d1, d2)
    if(transferTime == ''):
      break
    rowId += 1 
    prevServ = currServ
    # services_df = services_df.append(pd.Series(lst), ignore_index=True)
    services_df.loc[len(services_df)] = lst

# services_df.reset_index()

services_df.head(5)


,row_id,subject_id,hadm_id,transfertime,prev_serv,curr_serv
0,1,1,5815,12-06-2017 00:00:00,NONE,TRAUM
1,2,1,5815,10-06-2018 11:56:02,TRAUM,OBS
2,3,1,5815,05-29-2020 02:28:13,OBS,GU
3,4,2,5816,10-14-2035 00:00:00,NONE,MED
4,5,2,5816,01-10-2036 05:48:53,MED,GYN


In [10]:
services_df.to_csv('services_data.csv')